# CONNECTING TO `Payswift` DATABASE USING PYTHON

In [19]:
import psycopg2
import pandas as pd
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)

### CONNECTING TO PAYSWIFT (SQL DATABASE)

In [60]:
# connecting to Payswift database
conn = psycopg2.connect(
    dbname="Payswift", 
    user="postgres", 
    password="ray", 
    host="localhost")

cursor = conn.cursor()
print('connected to Payswift Database')

connected to Payswift Database


### INSERTING `transactions.csv` INTO `transactions` TABLE

In [83]:
# Load CSV and insert into transactions table
with open("../data/transactions.csv", "r") as f:
    cursor.copy_expert("COPY transactions FROM STDIN WITH CSV HEADER", f)
    
# commit the save
conn.commit()

print("Loaded 10K transactions into Postgres!")

Loaded 10K transactions into Postgres!


In [85]:
# Data Exploration
pd.read_sql("""
    SELECT * 
    FROM transactions
    LIMIT 10
""", conn)

,transaction_id,user_id,amount,timestamp,type,region,is_fraud
0,TXN00000,USR394,21282.63,2025-03-05 18:00:00,payment,Lagos,False
1,TXN00001,USR515,19434.87,2025-03-22 07:00:00,payment,Lagos,False
2,TXN00002,USR744,45570.23,2025-03-22 14:00:00,withdrawal,Nairobi,False
3,TXN00003,USR738,6485.54,2025-03-06 09:00:00,payment,Lagos,False
4,TXN00004,USR328,46007.09,2025-03-18 03:00:00,withdrawal,Nairobi,False
5,TXN00005,USR828,10224.54,2025-03-07 04:00:00,deposit,Nairobi,False
6,TXN00006,USR697,3419.21,2025-03-21 02:00:00,withdrawal,Nairobi,False
7,TXN00007,USR216,49340.21,2025-03-12 13:00:00,payment,Lagos,False
8,TXN00008,USR720,27888.67,2025-03-15 09:00:00,payment,Nairobi,False
9,TXN00009,USR978,27034.40,2025-03-27 23:00:00,withdrawal,Nairobi,False


In [84]:
# checking for fraud cases
pd.read_sql("""
    SELECT * 
    FROM transactions
""", conn)['is_fraud'].value_counts()

is_fraud
False    8699
True     1301
Name: count, dtype: int64

### INSERTING `users.csv` INTO `users` TABLE

In [49]:
# open users.csv and load into users table
with open("../data/users.csv", "r") as g:
    cursor.copy_expert("COPY users FROM STDIN WITH CSV HEADER", g)


# save load
conn.commit()

In [50]:
# load first 5 rows
pd.read_sql("""
    SELECT * 
    FROM users
    LIMIT 5
""", conn)

,user_id,account_age_days,avg_transaction_value
0,USR100,74,15532.97
1,USR101,357,14468.89
2,USR102,129,12657.60
3,USR103,48,7082.07
4,USR104,256,14871.66


## INSERTING `logins.csv` INTO `logins` TABLE

In [58]:
conn.rollback()  # Clears the failed transaction
with open("../data/logins.csv") as h:
    cursor.copy_expert("""
        COPY logins (user_id, timestamp, login_status, failed_attempts) 
        FROM STDIN 
        WITH CSV HEADER
    """, h)
conn.commit()

In [86]:
# load first 5 rows
pd.read_sql("""
    SELECT * 
    FROM logins
    LIMIT 5
""", conn, index_col='id')

,user_id,timestamp,login_status,failed_attempts
id,,,,
1,USR654,2025-03-02 09:00:00,success,0
2,USR769,2025-03-06 07:00:00,success,0
3,USR437,2025-03-29 14:00:00,success,0
4,USR462,2025-03-21 21:00:00,success,0
5,USR860,2025-03-28 09:00:00,success,0
